In [12]:
import numpy as np
import cv2
import pandas as pd

In [13]:
image_path  = 'data/driving_log.csv'
dt = pd.DataFrame(data=pd.read_csv(image_path))

In [14]:
dt.head(5)

,center,left,right,steering,throttle,brake,speed
0,IMG\center_2017_04_04_21_55_56_295.jpg,IMG\left_2017_04_04_21_55_56_295.jpg,IMG\right_2017_04_04_21_55_56_295.jpg,0.0,0.0,0,4.622093e-06
1,IMG\center_2017_04_04_21_55_56_444.jpg,IMG\left_2017_04_04_21_55_56_444.jpg,IMG\right_2017_04_04_21_55_56_444.jpg,0.0,0.0,0,9.887051e-07
2,IMG\center_2017_04_04_21_55_56_585.jpg,IMG\left_2017_04_04_21_55_56_585.jpg,IMG\right_2017_04_04_21_55_56_585.jpg,0.0,0.0,0,1.452095e-06
3,IMG\center_2017_04_04_21_55_56_732.jpg,IMG\left_2017_04_04_21_55_56_732.jpg,IMG\right_2017_04_04_21_55_56_732.jpg,0.0,0.0,0,2.705656e-06
4,IMG\center_2017_04_04_21_55_56_882.jpg,IMG\left_2017_04_04_21_55_56_882.jpg,IMG\right_2017_04_04_21_55_56_882.jpg,0.0,0.0,0,3.425475e-07


In [42]:
augmented_images , augmented_measurements = [] , []

def augment(image , measurement):
    augmented_images.append(np.fliplr(image))
    augmented_measurements.append(-measurement)
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    

In [43]:
#center_images=[]
#left_images =[]
#right_images=[]
#center_steering =[]
#left_steering =[]
#right_steering =[]


#images =[]
#measurements =[]
correction =0.2

for row in dt.values:
    #print (row[0])
    center_source_path = 'data/'+row[0]
    left_source_path = 'data/'+row[1]
    right_source_path = 'data/'+row[2]
    center_image = cv2.imread(center_source_path)
    left_image = cv2.imread(left_source_path)
    right_image = cv2.imread(right_source_path)
    #center_images.append(center_image)
    #left_images.append(left_image)
    #right_images.append(right_image)
    steering = row[3]
    #center_steering.append(steering)
    left_steering = (steering + correction)
    right_steering = (steering -correction)
    augment(center_image, steering)
    augment(left_image ,left_steering)
    augment(right_image, right_steering)
    #images.extend(center_image , left_image, right_image)
    #measurements.extend(center_steering , left_steering , right_steering)




In [44]:
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

In [45]:
X_train.shape

(16866, 160, 320, 3)

In [31]:
from keras.models import Sequential
from keras.layers import Flatten , Dense , Lambda , Cropping2D
from keras.layers import Convolution2D , MaxPooling2D , Dropout , Activation


model = Sequential()
model.add(Lambda(lambda x: x / 255.0 -0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,20), (0,0))))
model.add(Convolution2D(24,5,5 ,subsample=(2,2) , activation='relu'))
model.add(Convolution2D(36,5,5 ,subsample=(2,2) , activation='relu'))
model.add(Convolution2D(48,5,5 ,subsample=(2,2) , activation='relu'))

model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))

model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse' , optimizer='adam')
model.fit(X_train, y_train , validation_split=0.2, shuffle=True , epochs=6)

model.save('model.h5')

C:\Users\Praveen\Anaconda3\envs\SelfDrivingCar\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
C:\Users\Praveen\Anaconda3\envs\SelfDrivingCar\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
C:\Users\Praveen\Anaconda3\envs\SelfDrivingCar\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2), activation="relu")`
C:\Users\Praveen\Anaconda3\envs\SelfDrivingCar\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
C:\Users\Praveen\Anaconda3\envs\SelfDrivingCar\lib\site-packages\ipykernel\__main__.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


Train on 2248 samples, validate on 563 samples
Epoch 1/6
2248/2248 [==============================] - 31s - loss: 0.0596 - val_loss: 0.0309